#Imports

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

%cd gdrive/My Drive/3D Scene Understanding

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
[Errno 2] No such file or directory: 'gdrive/My Drive/3D Scene Understanding'
/content/gdrive/My Drive/3D Scene Understanding


In [ ]:
# set the matplotlib backend so figures can be saved in the background
import matplotlib
matplotlib.use("Agg")

# import the necessary packages
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, plot_confusion_matrix
from tensorflow.keras.models import Sequential,  load_model
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import SGD
import tensorflow as tf
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import argparse
import random
import pickle
import cv2
import os

#Prepare Data

In [ ]:
print("[INFO] loading dataset...")

names_csv = ['V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'W1', 'W2', 'W3', 'Y1', 'Y2', 'Y3', 'Class']
# grab the data and randomly shuffle them
dataset = pd.read_csv('dataset.csv', names=names_csv)
dataset.head()

dataset = dataset.iloc[1: , :]

for col in names_csv:
  dataset[col] = dataset[col].astype(np.int)

features = dataset.copy()
# shuffle dataset
# features = features.sample(frac=1).reset_index(drop=True)

labels = features.pop('Class')

features = np.array(features)
labels = np.array(labels)

[INFO] loading dataset...


#Construct Training Set and Test Set

In [ ]:
# partition the data into training and testing splits using 75% of
# the data for training and the remaining 25% for testing
(trainX, testX, trainY, testY) = train_test_split(features,
	labels, test_size=0.25, random_state=42)

In [ ]:
# convert the labels from integers to vectors (for 2-class, binary
# classification you should use Keras' to_categorical function
# instead as the scikit-learn's LabelBinarizer will not return a
# vector)
lb = LabelBinarizer()
trainY = lb.fit_transform(trainY)
testY = lb.transform(testY)

#Construct model

##Model

In [ ]:
# define the 3072-1024-512-3 architecture using Keras
model = Sequential()
model.add(Dense(25, input_shape=(12,), activation="sigmoid"))
model.add(Dense(len(lb.classes_), activation="softmax"))

##Optimizer and loss


In [ ]:
# initialize our initial learning rate and # of epochs to train for
INIT_LR = 0.1
EPOCHS = 80
# compile the model using SGD as our optimizer and categorical
# cross-entropy loss (you'll want to use binary_crossentropy
# for 2-class classification)
print("[INFO] training network...")
opt = SGD(lr=INIT_LR)
model.compile(loss="categorical_crossentropy", optimizer=opt,
	metrics=["accuracy"])

[INFO] training network...


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


#Fitting the model

In [ ]:
# train the neural network
H = model.fit(x=trainX, y=trainY, validation_data=(testX, testY),
	epochs=EPOCHS, batch_size=1)

Epoch 1/80
83/83 [==============================] - 1s 5ms/step - loss: 1.5149 - accuracy: 0.2289 - val_loss: 1.4577 - val_accuracy: 0.2500
Epoch 2/80
83/83 [==============================] - 0s 3ms/step - loss: 1.3554 - accuracy: 0.3494 - val_loss: 1.4538 - val_accuracy: 0.3929
Epoch 3/80
83/83 [==============================] - 0s 3ms/step - loss: 1.2446 - accuracy: 0.4699 - val_loss: 1.0379 - val_accuracy: 0.5714
Epoch 4/80
83/83 [==============================] - 0s 3ms/step - loss: 1.1411 - accuracy: 0.4578 - val_loss: 1.1112 - val_accuracy: 0.5357
Epoch 5/80
83/83 [==============================] - 0s 3ms/step - loss: 1.0282 - accuracy: 0.5542 - val_loss: 0.9557 - val_accuracy: 0.7500
Epoch 6/80
83/83 [==============================] - 0s 3ms/step - loss: 0.9721 - accuracy: 0.6145 - val_loss: 0.9387 - val_accuracy: 0.6786
Epoch 7/80
83/83 [==============================] - 0s 3ms/step - loss: 0.8811 - accuracy: 0.6024 - val_loss: 0.7985 - val_accuracy: 0.7500
Epoch 8/80
83/83 [==

#Evaluating the model

In [ ]:
# evaluate the network
print("[INFO] evaluating network...")
predictions = model.predict(x=testX, batch_size=1)

target_names=list(map(str,lb.classes_))
print(classification_report(testY.argmax(axis=1),
	predictions.argmax(axis=1), target_names=target_names))

# plot the training loss and accuracy
N = np.arange(0, EPOCHS)
plt.style.use("ggplot")
plt.figure()
plt.plot(N, H.history["loss"], label="train_loss")
plt.plot(N, H.history["val_loss"], label="val_loss")
plt.plot(N, H.history["accuracy"], label="train_acc")
plt.plot(N, H.history["val_accuracy"], label="val_acc")
plt.title("Training Loss and Accuracy (Simple NN)")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend()
plt.savefig('NN Model/accuracy_plot.png')


[INFO] evaluating network...
              precision    recall  f1-score   support

           1       1.00      1.00      1.00         7
           3       0.89      1.00      0.94         8
           5       1.00      0.82      0.90        11
           7       0.33      0.50      0.40         2

    accuracy                           0.89        28
   macro avg       0.81      0.83      0.81        28
weighted avg       0.92      0.89      0.90        28



#Saving the model

In [ ]:
# save the model and label binarizer to disk
print("[INFO] serializing network and label binarizer...")
model.save('NN Model/nn.model', save_format="h5")
f = open('NN Model/nn_lb.pickle', "wb")
f.write(pickle.dumps(lb))
f.close()

[INFO] serializing network and label binarizer...


#Predicting using the model

In [ ]:
# load the model and label binarizer
print("[INFO] loading network and label binarizer...")
model = load_model('NN Model/nn.model')
lb = pickle.loads(open('NN Model/nn_lb.pickle', "rb").read())

[INFO] loading network and label binarizer...


In [ ]:
# make a prediction on the vector
data = [0, 0, 0, 0, 3, 0, 1, 0, 3, 0, 1, 1]
data = np.expand_dims(data, axis=0)
preds = model.predict(data, batch_size=1)
# find the class label index with the largest corresponding
# probability

i = preds.argmax(axis=1)
label = lb.classes_[i]

print('Le vecteur de sortie est : ', preds, '\n')
print('L\'objet est de la classe : ', label[0])

Le vecteur de sortie est :  [[2.7597413e-04 3.7278078e-02 2.3041721e-01 7.3202872e-01]] 

L'objet est de la classe :  7
